# Open Neural Network Exchange  (**ONNX**)
![ONNX](ONNX-Logo.svg)
ONNX – открытый стандарт для конвертации моделей машинного обучения из разных фреймворков в единый формат, а также для обмена моделями между фреймворками

Пример сохранения модели ElKulako/cryptobert в onnx формате и оценка измнения скорости инференса в PyTorch и ONNX.


[Ссылка на ElKulako/cryptobert модель на Huggingface ](https://huggingface.co/ElKulako/cryptobert)


In [1]:
import numpy as np
import os, time
from transformers import TextClassificationPipeline, AutoTokenizer, AutoModelForSequenceClassification 
import torch, onnx
import onnxruntime
from onnxruntime import InferenceSession, SessionOptions
from onnxruntime.capi.onnxruntime_pybind11_state import InvalidArgument

1. Используем библиотеку transformers от HuggingFace для работы с моделью
Флаг FLAG_SOFT_USE == True : Преобразуем logits выходы нейронной сети в оценки настроения новости с 
                             помощью функции Softmax
Флаг FLAG_SOFT_USE == False : Функция Softmax  не применяется 

In [2]:
FLAG_SOFTMAX_USE = False

In [3]:
model_name = "ElKulako/cryptobert"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast = False)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = 3)
pipeline =  TextClassificationPipeline(model=model, tokenizer=tokenizer, max_length=64, truncation=True, padding='max_length', top_k=None)

In [4]:
t_start = time.time()
news_list = ["How the GBTC premium commerce ruined Barry Silbert, his DCG empire and took crypto lending platforms with them",
             "MicroStrategy’s Bitcoin Strategy Sets Tongues Wagging Even As It Doubles Down On BTC Purchases ⋆ ZyCrypto",
             "How the GBTC premium trade ruined Barry Silbert, his DCG empire and took crypto lending platforms with them",
             "Bitcoin Holders To Expect More Difficulties As Data Point To Looming BTC Price Drop",
             "Bitcoin Breaks Past $17,000 Barrier – Will BTC Also Breach 4% Weekly Run?",
             "Bitcoin Price Today 9 Jan: BTC Increases By 1.79% Painting The Chart Green",
             "Bitcoin: This is what large investor and retail interest can do for BTC over time"                          
]
pytorch_output = []
for n in news_list:
    if FLAG_SOFTMAX_USE:
        pytorch_output.append(pipeline(n))
    # Результат работы модели без преобразования  logits функцией softmax
    else:        
         input_ids = torch.tensor(tokenizer.encode(n)).unsqueeze(0)
         pytorch_output.append(model(input_ids))
t_torch = time.time() - t_start
    
print("{} новостей обработано за {:.7f}с {} ".format(len(news_list), t_torch, ("с softmax"  if FLAG_SOFTMAX_USE else "без softmax")))   


7 новостей обработано за 0.6470144с без softmax 


2. Конвертируем Cryptobert модель в onnx-формат и сохраняем в файл bert.onnx

In [5]:
dummy_model_input = tokenizer(
        "текст для конвертации",
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt",
    ).to("cpu")
torch.onnx.export(
    model,
    dummy_model_input["input_ids"],
    "bert.onnx",
    opset_version=12,
    input_names=["input_ids"],
    output_names=["output"],
    dynamic_axes={
        "input_ids": {
            0: "batch_size",
            1: "sequence_len"
        },
        "output": {
            0: "batch_size"
        }
    }
)

3. Выполняем инференс с помощью onnxruntime
ONNX Runtime – библиотека для кроссплатформенного ускорения обучения и инференса моделей;

In [6]:
options = SessionOptions()
options.graph_optimization_level = onnxruntime.GraphOptimizationLevel.ORT_ENABLE_ALL
options.intra_op_num_threads = 1
session = InferenceSession("bert.onnx", options, providers=["CPUExecutionProvider"])
session.disable_fallback()


In [7]:
def softmax(logits):
    logits = np.exp(logits)
    s = sum(logits)
    for i in range(3):
        logits[i]=logits[i]/s
    return logits

In [8]:
t_start = time.time()
onnx_output = []
for n in news_list:   
    inputs = tokenizer(n)    
    input_feed = {"input_ids":np.array(inputs["input_ids"]).reshape(1,-1)}
    try:
        output = session.run(
             output_names=["output"],
             input_feed=input_feed
        )[0][0]
        if FLAG_SOFTMAX_USE:
            output = softmax(output)            
        onnx_output.append(output)
    except (RuntimeError, InvalidArgument) as e:
        print(e) 
  
   # str ="{:.6f}, {:.6f}, {:.6f}".format(output[0], output[1], output[2])) after softmax
t_onnx = time.time() - t_start
print("{} новостей обработано за {}c {}".format(len(news_list), t_onnx, ("с softmax"  if FLAG_SOFTMAX_USE else "без softmax") ))


7 новостей обработано за 0.5392591953277588c без softmax


Оценим время инференса Pytorch и ONNX моделей. 

In [9]:
print("Результат работы модели {} ".format("с softmax"  if FLAG_SOFTMAX_USE else "без softmax"))
print("Onnx модель работает на {:.6f}c быстрее, то есть в {:.2f} раз быстрее : onnx {:.6f}c против pytorch {:.6f}c".format(t_torch-t_onnx, t_torch/t_onnx, t_onnx, t_torch ))

Результат работы модели без softmax 
Onnx модель работает на 0.107755c быстрее, то есть в 1.20 раз быстрее : onnx 0.539259c против pytorch 0.647014c


Проверим результа работы инфренса Pytorch и ONNX

In [10]:
if FLAG_SOFTMAX_USE:
    print("C softmax")
   
    pytorch_output_ch = []
    for i in range(len(pytorch_output)):
        for j in range(3):
            a = pytorch_output[i][0][j]
            if a["label"] == "Neutral":
               a2 = a["score"]
            if a["label"] == "Bullish":
               a3 = a["score"]
            if a["label"] == "Bearish":
               a1 = a["score"]
        pytorch_output_ch.append((a1,a2,a3))   
    
else:
    print("Logits (без oftmax)")
    pytorch_output_ch = []
    for i in range(len(pytorch_output)):
        a = pytorch_output[i].logits.detach().numpy()[0]
        pytorch_output_ch.append((a[0],a[1],a[2]))
    
b = list(zip(pytorch_output_ch,onnx_output)) 
[print(i) for i in b];

Logits (без oftmax)
((-4.98122, 2.5637887, 1.7561779), array([-4.9812217,  2.5637898,  1.7561791], dtype=float32))
((-3.2925947, 1.5659943, 1.4243592), array([-3.2925928,  1.5659933,  1.4243565], dtype=float32))
((-4.9118767, 2.8916512, 1.3431506), array([-4.9118824,  2.8916552,  1.343152 ], dtype=float32))
((-4.230209, 1.903221, 1.9884388), array([-4.2302027,  1.9032187,  1.988435 ], dtype=float32))
((-2.0214012, 1.0943921, 1.3000424), array([-2.0214126,  1.0943971,  1.3000479], dtype=float32))
((-3.1212604, 1.6515594, 1.3943415), array([-3.121268 ,  1.6515632,  1.3943424], dtype=float32))
((0.09347219, 0.21490246, -0.15068218), array([ 0.09346019,  0.2149083 , -0.15067741], dtype=float32))


Результат работы модели без softmax совпадает до 5 знака после запятой.